#### From time predictions to location predictions

We will try predicting the road condition in 1 meter intervals.

In [33]:
# Create an example of weights for simplicity
# We have car A, B and C which have all been driving over a segment. We are interested in

# Simple exmaple
A_pred = 5
A_speed = 50 

B = 7
B_speed = 30

C = 6 
C_speed = 40


In [34]:
def time_to_drive_two_meters(speed_kmh):
    # Convert speed from km/h to m/s
    speed_ms = speed_kmh * (1000 / 3600)
    
    # Distance to be traveled in meters
    distance_m = 10
    
    # Calculate time in seconds
    time_seconds = distance_m / speed_ms
    
    return time_seconds

# Example usage
speed_kmh = 100  # Speed in km/h
time_seconds = time_to_drive_two_meters(speed_kmh)
print(f"Time to drive 2 meters at {speed_kmh} km/h: {time_seconds} seconds")


Time to drive 2 meters at 100 km/h: 0.36 seconds


In [35]:
# Use the speed to estimate the amount of sample points there are in the area of interest (AOI)
# The AOI is thought to be defined by an X meter long segment
sample_freq = 250  # Sample frequency in Hz
A_samples = time_to_drive_two_meters(A_speed) * sample_freq
B_samples = time_to_drive_two_meters(B_speed) * sample_freq
C_samples = time_to_drive_two_meters(C_speed) * sample_freq
total_samples = A_samples + B_samples + C_samples

A_weight = A_samples / total_samples
B_weight = B_samples / total_samples
C_weight = C_samples / total_samples


In [36]:
# Create a function that map each location to a 1 second time windows.
AIO_lan_lon = [[1, 2], [3, 4], [5, 6]] # Example of locations
loc_cars = {}
cars = ["car_1", "car_2", "car_3"] 

for car in cars:
    loc_cars[car] = []
    # go trough each location and map it to a 1 second time window
    for lan, lon in AIO_lan_lon:
        found_windown = 1
        loc_cars[car].apend(found_windown)
        



3

In [38]:
# We will now localise the point of interest in the time domains.
# It does not work to just take the first and last lon. and lat. and not the median either. 
# We need to be able to find the location fast, right now we will just 

# Should they all have a dictionary?
A_t1_data = [[1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10]]
A_t1_pred = 5
